In [1]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from IPython.display import Audio
import webrtcvad
import wave
import speech_recognition as sr           # package SpeechRecognition
import datetime
import strsimpy
import xml.etree.ElementTree as ET
import soundfile as sf
from pydub import AudioSegment
from pydub.silence import split_on_silence
import os
import scipy
from scipy import ndimage
import spleeter
import subprocess

In [2]:
# loading signal
def load_signal(file, samplerate=16000):
    """
    Parameters:
        string file : chemin du fichier audio
        int samplerate : fréquence d'échantillonage
    Returns:
        bytes binary_data : signal sous forme binaire
        numpy.ndarray signal : signal
        int fe : fréquence d'échantillonage
    """
    
    signal, fe = librosa.load(file, sr=samplerate, mono=True)
    
    w = wave.open(file, "rb")
    binary_data = w.readframes(w.getnframes())
    
    return binary_data, signal, fe

# speech detection using webrtcvad
def get_speech_intervals(bin_signal, samplerate=16000, agg=3, smooth='rolling_mean', affichage=False):
    """
    Parameters:
        bytes bin_signal : signal sous forme binaire
        int samplerate : fréquence d'échantillonage
        int agg : paramètre de la fonction vad (1/2/3)
        str smooth : méthode de lissage à utiliser
        bool affichage : booléen : afficher ou non un graphique représentant le résultat
    Returns:
        Series speech : signal booléen indiquant où se trouve la parole dans le signal d'entré
    """
    
    if smooth is None : smooth = 'rolling_mean'
    
    recomposed_signal = np.frombuffer(bin_signal, dtype=np.int16)

    vad = webrtcvad.Vad()
    vad.set_mode(mode=agg)

    millisec = 10
    fenetre = int(samplerate * millisec * 2 / 1000)

    fen_speech = [vad.is_speech(bin_signal[m:m+fenetre], samplerate) 
                  for m in range(0,len(bin_signal),fenetre)
                 if len(bin_signal[m:m+fenetre]) == fenetre]

    widened_speech = np.array([[s] * int(fenetre/2) for s in fen_speech]).ravel()
    speech = np.full(len(recomposed_signal), widened_speech[-1])
    speech[0:len(widened_speech)] = widened_speech

    # adoucissement
    if smooth == 'closing':
        smoothed_speech = ndimage.binary_closing(input=speech, structure=np.array([1]*10000)).astype(bool)
    elif smooth == 'rolling_mean' :
        smoothed_speech = pd.Series(speech).rolling(window=int(samplerate/5), min_periods=2, center=True).mean() > 0.2
    
    if affichage :
        plt.figure(figsize=(14,4))
        plt.plot(np.arange(len(smoothed_speech))/samplerate, smoothed_speech*max(recomposed_signal), label="speech")
        plt.plot(np.arange(len(recomposed_signal))/samplerate, recomposed_signal, label="signal")
        plt.xlabel("sec")
        plt.legend(loc=4)
        plt.show()
        
    return pd.Series(smoothed_speech)

# SNR using speech detection (bad on recomposed signal from binary)
def get_SNR(signal, speech, samplerate=16000):
    """
    Parameters:
        numpy.ndarray signal : signal audio
        Series speech : signal booléen indiquant où se trouve la parole dans le signal d'entré
        int samplerate : fréquence d'échantillonage
    Returns:
        float SNR : donne le rapport de l'energie du signal parlé sur l'énergie totale
    """
    s = pd.Series(signal)
    energy_total = (s**2).rolling(window=samplerate, min_periods=2, center=True).mean()

    energy_speech = (s[speech]**2).rolling(window=samplerate, min_periods=2, center=True).mean()
    energy_notSpeech = (s[~speech]**2).rolling(window=samplerate, min_periods=2, center=True).mean()

    SNR = energy_speech.mean() / (energy_speech.mean() + energy_notSpeech.mean())
    
    return SNR

# a partir des timestamps des sous-titres, creer un signal booléen représantant la parole
def get_filtre_paroles(ts, dilation_ms=100):
    """
    Parameters:
        list ts : liste des phrases, timestamps et durée des fichier xml
        int dilation_ms : ajout de quelques millisecondes avant et après les timestamps
    Returns:
        numpy.ndarray paroles : signal booléen indiquant où se trouve la parole dans le signal d'entré
    """
    # creation du filtre paroles
    ts_longueur = int(np.sum(ts[-1][0])*1000)
    paroles = np.zeros(ts_longueur)
    for i,v in ts :
        a,d = int(i[0]*1000), int(i[1]*1000)
        paroles[a:a+d] = 1
    
    paroles = ndimage.binary_dilation(input=paroles, structure=np.array([1]*dilation_ms)).astype(bool)
    return paroles

# filtre à la main pour estimer quand les sous-titres commencent
def suppression_bords(m, t):
    """
    Parameters:
        m : array d'une certaine longeur représentant les décallages possibles des sous-titres
        t : decallage à la main de t pas
    Returns:
        Series : représente un score pour chaque décallage
    """
    
    droite = np.zeros([len(m)+2*t])
    droite[0:t], droite[-t:], droite[t:-t] = 4,4,1
    droite = droite / droite.sum()
    m2 = pd.Series(droite).rolling(window=2*t+1, center=True).mean()[t:-t]
    m2.index = m.index
    return m2/2

# a partir d'un signal audio et d'un signal booléen représentant la parole,
# calcul d'aires du signal parlé et non parlé en fonction d'un décallage des sous-titres
# pour chaque décallage retourne une métrique basé sur ces calculs d'aires
def get_decalage(paroles, signal, pas=100, affichage=False):
    """
    Parameters:
        paroles : signal booléen indiquant parlé / non parlé dans les sous-titres
        numpy.ndarray signal : signal
        int pas : on décalle au fur et à mésure les sous titres de pas millisecondes
        bool affichage : afficher ou non un graphique représentant le résultat
    Returns:
        Series metric : représente un score pour chaque décallage
    """
    
    intervals = range(0, len(signal) - len(paroles), pas)
    aires_0 = np.zeros([len(intervals)])
    aires_1 = np.zeros([len(intervals)])

    for i,debut in enumerate(intervals):
        aire_0 = signal[debut:debut+len(paroles)] * (paroles == 0)
        aire_1 = signal[debut:debut+len(paroles)] * (paroles == 1)
        aires_0[i] = abs(aire_0).sum()
        aires_1[i] = abs(aire_1).sum()
    
    if len(aires_0) > 0 : decalage = np.argmin(aires_0 - aires_1)*pas
    else : decalage = 0
    
    aire_0 = signal[decalage:decalage+len(paroles)] * (paroles == 0)[:len(signal)]
    aire_1 = signal[decalage:decalage+len(paroles)] * (paroles == 1)[:len(signal)]

    resultat = np.zeros(len(signal))
    resultat[decalage : decalage + len(paroles)] = paroles[0:len(paroles)]
    
    metric = (aires_0-aires_1)
    metric = metric / abs(metric.sum())
    metric = pd.Series(metric, index=np.arange(0,len(metric))*pas)
    if len(metric) == 0 : metric = pd.Series(np.zeros(1))
    
    if affichage:
        plt.figure(figsize=(14,2))
        plt.title(str(decalage))
        plt.plot(resultat, label="paroles")
        plt.plot(signal)
        plt.legend(loc=2)
        plt.show()

        plt.figure(figsize=(14,2))
        plt.plot(metric, label="aires")
        plt.legend(loc=2)
        plt.show()
        
    return metric

# return text sentence from audio
def get_recognition(audiofile, adjust_noise=False):
    """
    Parameters:
        str audiofile : chemin du fichier audio
        bool adjust_noise : utiliser ou non adjust_for_ambient_noise
    Returns:
        float conf : indice de confiance de google pour ce qui est reconnu
        str text : texte reconnu
    """
    r  = sr.Recognizer()
    demo = sr.AudioFile(audiofile)
    with demo as source:
        if adjust_noise: r.adjust_for_ambient_noise(source)
        audio = r.record(source)
    try:
        recon = r.recognize_google(audio, language='fr-FR', show_all=True)
        if len(recon) == 0 :
            text = ''
            conf = 0
        else :
            text = recon['alternative'][0]['transcript']
            conf = recon['alternative'][0]['confidence']
    except LookupError:
        print("LookupError : Could not understand audio")
        text = ''
        conf = 0
    
    return conf, text

# return sentences, when they start in seconds and their length in seconds
def get_timed_sentences(xmlfile):
    """
    Parameters:
        str xmlfile : chemin du fichier xml
    Returns:
        list sentences : liste de valeurs (debut (sec), duree (sec), phrase (texte))
    """
    tree = ET.parse(xmlfile)

    sentences = [([e.attrib['value'] for e in sent if e.tag=='time'],
                  " ".join([w.text.strip() for w in sent if w.text is not None]))
                 for sent in tree.getroot()]

    t0 = datetime.datetime.strptime(sentences[0][0][0], '%H:%M:%S,%f')
    for i,(t,s) in enumerate(sentences):
        t1 = datetime.datetime.strptime(t[0], '%H:%M:%S,%f')
        t2 = datetime.datetime.strptime(t[1], '%H:%M:%S,%f')
        sentences[i] = (((t1 - t0).total_seconds() , (t2 - t1).total_seconds()), s)

    return sentences

# split the speech signal (0/1) into several signal (1)
def split_speech(speech, samplerate=16000, sec_before=1, sec_after=0.5):
    """
    Parameters:
        Series speech : chemin du fichier xml
        int samplerate : fréquence d'échantillonage
        float sec_before : ajouter x secondes avant chaque début de parlé
        float sec_after : ajouter x secondes après chaque fin de parlé
    Returns:
        DataFrame df_intervals : dataframe qui pour chaque ligne contient 5 colonnes:
            add_noise : temps de bruit à ajouter (ou non)
            start_signal : début de la séquence parlé (avec ajout ou non de temps avant)
            start_speech : début de la parole
            end_speech : fin de la parole
            end_signal : fin de la séquence parlé (avec ajout ou non de temps après)
    """
    f = np.array([1,-1])
    r = np.convolve(speech, f, 'same')
    starts, ends = np.where(r == 1)[0].tolist(), np.where(r == -1)[0].tolist()
    if len(ends) == 0 : ends = [len(speech)]
    if ends[0] < starts[0] : starts = [0] + starts
    if len(starts) > len(ends) : ends = ends + [len(speech)]

    speech_intervals = np.array([(0,
                                  starts[i] - int(samplerate*sec_before),
                                  starts[i],
                                  ends[i], ends[i] + int(samplerate*sec_after))
                                 for i in range(len(starts))])

    diffs = [(v[2] - speech_intervals[i][-2] - int(samplerate*sec_before)) for i,v in enumerate(speech_intervals[1:])]
    diffs = [0 if speech_intervals[0,1] > 0 else -speech_intervals[0,1]] + [0 if v > 0 else abs(v) for v in diffs]

    speech_intervals[:,0] = diffs
    speech_intervals[:,1] += speech_intervals[:,0]
    speech_intervals[speech_intervals < 0] = 0

    df_intervals = pd.DataFrame(speech_intervals,
                                columns=['add_noise','start_signal','start_speech','end_speech','end_signal'])
    return df_intervals

# return bool serie where the background noise is
def get_background(signal, speech, samplerate, affichage=False):
    """
    Parameters:
        numpy.ndarray signal : signal audio
        Series speech : signal booléen
        samplerate : fréquence d'échantilonnage
        affichage : affichage ou non d'un graphique représentant le résultat
    Returns:
        Series background : signal booléen indiquant ou est le signal de fond
    """
    background = speech.rolling(window=int(samplerate/4), center=True).mean() < 1/10
    
    if affichage:
        plt.figure(figsize=(14,4))
        plt.plot(signal, label="signal")
        plt.plot(speech * max(signal) / 2, label="speech")
        plt.plot(background * max(signal), label="noise")
        plt.legend(loc=4)
        plt.show()
        
    return background

# make a noise from signal and background
def make_noise(signal, background, samplerate, lenght=3):
    """
    Parameters:
        numpy.ndarray signal : signal audio
        Series background : signal booléen
        int samplerate : fréquence d'échantilonnage
        int lenght : temps en secondes du bruit voulu
    Returns:
        numpy.ndarray noise : signal du bruit
    """
    if background.sum() < samplerate/10 :
        noise = signal[:len(signal) - len(signal)%samplerate].reshape(samplerate, int(len(signal)/samplerate)).mean(axis=1)
    else :
        noise = np.array([signal[background][m:m+int(lenght*samplerate)]
                          for m in range(len(signal[background][::int(lenght*samplerate)]))]).mean(axis=0)
        if len(noise) < lenght*samplerate:
            noise = np.array(list(noise) * (int(lenght*samplerate / len(noise))+1))[0:int(lenght*samplerate)]
    return noise

# recherche la valeur en dB de coupure entre silence et bruit
# en fonction d'un nombre de coupure du signal voulu
def get_dB_treshold(dB, n_win_wanted=11, affichage=False):
    """
    Parameters:
        numpy.ndarray dB : array des dB d'un signal
        int n_win_wanted : nombre de séparation de dB recherché
        bool affichage : représenter ou non les résultats
    Returns:
        int treshold : valeur du dB qui sépare silence et non silence
    """
    dBs = list(range(50,100))
    res = pd.Series(0, index=dBs)

    for v_dB in dBs:
        d = np.array(dB) + v_dB
        r = (d > 0)

        f = np.array([1,1]) / 2
        c = (np.convolve(r,f,'same') < 0.8)

        f2 = np.array([-1,1])
        c2 = np.convolve(c,f2,'same')
        res[v_dB] = sum(c2==max(c2))


    treshold = min(list(res.items()), key=lambda x: abs(x[1]-n_win_wanted))[0]

    d = np.array(dB) + treshold
    r = (d > 0)
    f = np.array([1,1]) / 2
    c = (np.convolve(r,f,'same') < 0.8)
    f2 = np.array([-1,1])
    c2 = np.convolve(c,f2,'same')

    if affichage :
        plt.figure(figsize=(14,3))
        plt.title("treshold : " + str(treshold))
        plt.plot(d, alpha=0.8)
        plt.plot(c2 * max(d), alpha=0.8)
        plt.show()
    return treshold

# segemente l'audio suivant plusieurs méthodes
def get_chunks(audiofile, samplerate=16000, method='subtitles', smooth='closing', speech=None, subtitles=None):
    """
    Parameters:
        str audiofile : chemin du fichier audio
        int samplerate : fréquence d'échantillonage
        str method : méthode de segmentation de l'audio
        str smooth : méthode de lissage à utiliser dans get_speech_intervals
        Series speech : signal booléen de la parole calculé par vad
        numpy.ndarray subtitles : signal booléen de la parole à partir des sous-titres
    Returns:
        list listenable_chunks : liste de segments audio
    """
    song = AudioSegment.from_wav(audiofile)
    
    if method == 'silence': 
        # split track where silence is quieter than the threshold in dBFS for x milliseconds
        fen = 50
        n_win_wanted = round(len(song) / 1000 / 3) # on vise des chunk de 3 secondes en moyenne
        dB = np.array([song[i:i+fen].dBFS for i in range(0,len(song),fen)])
        treshold = get_dB_treshold(dB, n_win_wanted=n_win_wanted, affichage=False)
        chunks = split_on_silence(song, min_silence_len=100, silence_thresh=-treshold)
    
    elif method == 'speech':
        # split track based on speech detection
        bin_signal, signal, fe = load_signal(audiofile, samplerate=samplerate)
        if speech is None :
            speech = get_speech_intervals(bin_signal, samplerate=fe, agg=3, smooth=smooth, affichage=False)
        df_intervals = split_speech(speech, sec_before=0.8, sec_after=0.3)
        chunks_sep = [(int(df_intervals.loc[i,'start_speech']/samplerate*1000),
                       int(df_intervals.loc[i,'end_speech']/samplerate*1000))
                      for i in range(len(df_intervals))]
        chunks = [song[cs[0]:cs[1]] for cs in chunks_sep]
        
    elif method == 'stable_cuts':
        # split every x secondes
        bin_signal, signal, fe = load_signal(audiofile, samplerate=samplerate)
        df_intervals = pd.DataFrame(columns=['add_noise','start_signal','start_speech','end_speech','end_signal'])
        cuts = pd.Series(signal)[::samplerate*5].index
        for i,ci in enumerate(cuts):
            df_intervals.loc[i,'add_noise'] = samplerate
            df_intervals.loc[i,'start_signal'] = ci
            df_intervals.loc[i,'start_speech'] = ci
            df_intervals.loc[i,'end_speech'] = min(len(signal), ci+samplerate*10)
            df_intervals.loc[i,'end_signal'] = min(len(signal), ci+samplerate*10)
        chunks_sep = [(int(df_intervals.loc[i,'start_speech']/samplerate*1000),
                       int(df_intervals.loc[i,'end_speech']/samplerate*1000))
                      for i in range(len(df_intervals))]
        chunks = [song[cs[0]:cs[1]] for cs in chunks_sep]
        
    elif method == 'subtitles':
        # use subtitles timestamp to split the song, with help from speech detection
        bin_signal, signal, fe = load_signal(audiofile, samplerate=samplerate)
        if speech is None :
            speech = get_speech_intervals(bin_signal, samplerate=fe, agg=3, smooth=smooth, affichage=False)
        
        # resample audio to look like the song variable
        signal_rs = scipy.signal.resample(np.array(signal), round(len(np.array(signal)) / samplerate * 1000))
        speech_rs = scipy.signal.resample(np.array(speech), round(len(np.array(speech)) / samplerate * 1000)) > 0.5
        
        # subtitles length do not match audio length, find where to start:
        pas = 100
        m0 = get_decalage(subtitles, signal_rs, pas=pas, affichage=False)
        m1 = get_decalage(subtitles, speech_rs, pas=pas, affichage=False)
        m2 = suppression_bords(m0, t=5)
        decalage = np.argmin(np.array(m1+m2)) * pas
        
        filtre = np.zeros(len(song))
        filtre[decalage : decalage + len(subtitles)] = subtitles[0:len(song)]
        
        # split the single audio input into several speech audio output
        df_intervals = split_speech(filtre, samplerate=1000, sec_before=0.5, sec_after=0.5)
        chunks_sep = [(int(df_intervals.loc[i,'start_signal']),
                       int(df_intervals.loc[i,'end_signal']))
                      for i in range(len(df_intervals))]
        chunks = [song[cs[0]:cs[1]] for cs in chunks_sep]
    
    listenable_chunks = [c for c in chunks if len(c) > 300]
    return listenable_chunks

# write and open a generated noise from a audio file
def make_write_get_noise(audiofile, samplerate=16000, duration=500, smooth='closing', speech=None):
    """
    Parameters:
        str audiofile : chemin du fichier audio
        int samplerate : fréquence d'échantillonage
        int duration : durée du bruit en millisecondes
        str smooth : méthode de lissage à utiliser dans get_speech_intervals
        str speech : signal booléen de la parole calculé par vad
    Returns:
        AudioSegment noise : segment audio
    """
    bin_signal, signal, fe = load_signal(audiofile, samplerate=samplerate)
    if speech is None :
        speech = get_speech_intervals(bin_signal, samplerate=fe, agg=3, 
                                      smooth=smooth, affichage=False)
    background = get_background(signal, speech, fe, affichage=False)
    noise = make_noise(signal, background, fe, lenght=duration/1000)
    filename = './audio_chunks/noise.wav'
    sf.write(filename, noise, samplerate=fe, subtype='PCM_16')
    noise = AudioSegment.from_wav(filename)
    return noise

# text from audio file using google recognizer
def recon(audiofile, samplerate=16000, method='speech', smooth='closing', speech=None, subtitles=None, noise=False):
    """
    Parameters:
        str audiofile : chemin du fichier audio
        int samplerate : fréquence d'échantillonage
        str method : méthode de segmentation de l'audio
        str smooth : méthode de lissage à utiliser dans get_speech_intervals
        Series speech : signal booléen de la parole calculé par vad
        numpy.ndarray subtitles : signal booléen de la parole depuis les sous-titres
        bool noise : ajouter ou non du bruit
    Returns:
        float overall_conf : moyenne de la confiance que google a dans la reconnaissance de tous les chunks
        str overall_text : texte représentant l'ensemble de ce qui est reconnu dans tous les chunks
    """
    # create chunks
    listenable_chunks = get_chunks(audiofile, samplerate=samplerate,
                                   method=method, smooth=smooth, speech=speech, subtitles=subtitles)
    
    # noise and silence creation
    chunk_noise  = make_write_get_noise(audiofile, samplerate=samplerate, duration=500, smooth=smooth, speech=speech)
    chunk_silent = AudioSegment.silent(duration=500)

    # create a directory to store the audio chunks.
    try: os.mkdir('./audio_chunks') 
    except(FileExistsError): pass
    
    overall_conf = []
    overall_text = ''
    for i,chunk in enumerate(listenable_chunks):
        
        # add 500 milliseconds of silence or noise and raise the volume by 16 dB
        if noise : audio_chunk = chunk_noise +  (chunk + 16) + chunk_noise
        else :     audio_chunk = chunk_silent + (chunk + 16) + chunk_silent
        
        # save the newly created chunk
        filename = './audio_chunks/chunk'+str(i)+'.wav'
        audio_chunk.export(filename, bitrate ='192k', format ="wav") 

        # recognize the chunk
        r = sr.Recognizer()
        with sr.AudioFile(filename) as source: 
            audio_listened = r.record(source) 
        try:
            rec = r.recognize_google(audio_listened, language='fr-FR', show_all=True)
            if len(rec) == 0 : 
                conf = 0
                text = ''
            else :
                if 'confidence' in rec['alternative'][0]: conf = rec['alternative'][0]['confidence']
                else : conf = np.nan
                text = rec['alternative'][0]['transcript']
            
            overall_text += ' ' + text
            overall_conf.append(conf)
        except sr.UnknownValueError: print("-- Could not understand audio") 
        except sr.RequestError as e: print("-- Could not request results. check your internet connection")
    
    overall_text = " ".join(overall_text.split())
    overall_conf = np.array(overall_conf)[np.array(overall_conf) != 0].mean()
    
    return overall_conf, overall_text

# get features from a scene
def extract_features(scene, method='subtitles', smooth='closing', noise=False):
    """
    Parameters:
        str scene : nom de la scène (ex : '100_1')
        str method : méthode de segmentation de l'audio
        str smooth : méthode de lissage à utiliser dans get_speech_intervals
        bool noise : ajouter ou non du bruit
    Returns:
        float SR : proportion qui est parlée dans la scène
        float SNR : rapport signal sur bruit
        float VBR : rapport vocal on background
        float conf : indice de confiance de google
        str text : texte reconnu par google
        float score : score de reconnaissance basé sur la distance de Levenshtein
        str sent : texte à reconnaitre
    """
    spleet(scene)
    audiofile = spleeter_output_dir + scene + audio_extension.split('.')[-2] + '/vocals_resample.wav'
    xmlfile   = dir_texte + scene + texte_extension
    
    # load data
    bin_signal, signal, fe = load_signal(audiofile, samplerate=samplerate)
    
    # speech intervals à partir de la librairie webrtcvad
    speech = get_speech_intervals(bin_signal, samplerate=fe, agg=3, smooth=smooth, affichage=False)
    SR = speech.sum()/len(speech)
    
    # SNR (from speech data)
    SNR = get_SNR(signal, speech, samplerate=fe)
    
    # VBR (vocal background ratio from speeted file)
    vocal_file = spleeter_output_dir + scene + audio_extension.split('.')[-2] + '/vocals.wav'
    backg_file = spleeter_output_dir + scene + audio_extension.split('.')[-2] + '/accompaniment.wav'
    signal_vocal, fe_vocal = librosa.load(vocal_file, sr=samplerate, mono=True)
    signal_backg, fe_backg = librosa.load(backg_file, sr=samplerate, mono=True)
    VBR = (signal_vocal**2).sum() / ((signal_vocal**2).sum() + (signal_backg**2).sum())
    
    # xml to text
    sentences = get_timed_sentences(xmlfile)
    sent_ponct = " ".join([s[1] for s in sentences])
    sent = " ".join("".join([x if x.isalpha() else " " for x in sent_ponct]).split())
    
    # speech intervals à partir des sous titres
    subtitles = get_filtre_paroles(sentences, dilation_ms=300)
    
    # audio to text
    conf, text = recon(audiofile, samplerate=fe, method=method, smooth=smooth,
                       speech=speech, subtitles=subtitles, noise=noise)
    
    # distance between text_audio and text_xml and score
    levenshtein = strsimpy.Levenshtein()
    distance = levenshtein.distance(text.lower(), sent.lower())
    score = distance / len(sent)
    
    return SR, SNR, VBR, conf, text, score, sent

# generate the features dataframe
def make_audio_features(scenes, df=None, samplerate=16000, method='silence', smooth='rolling_mean', noise=False):
    """
    Parameters:
        str scenes : liste de scènes (ex : ['100_1','101_7'])
        DataFrame df : dataframe des features (déja calculé)
        int samplerate : fréquence d'échantillonage
        str method : méthode de segmentation de l'audio
        str smooth : méthode de lissage à utiliser dans get_speech_intervals
        bool noise : ajouter ou non du bruit
    Returns:
        DataFrame df : dataframe des features
    """
    if df is None : df = pd.DataFrame(columns=['SCENE','SR','SNR','VBR','CONF','RECON','SCORE','XML'])
    for scene in scenes:
        if scene not in list(df['SCENE']):
            SR, SNR, VBR, conf, text, score, sent = extract_features(scene, method=method, smooth=smooth, noise=noise)
            
            row = pd.Series([scene,SR,SNR,VBR,conf,text,score,sent],
                            index = df.columns)

            df = df.append(row, ignore_index=True)
            print(scene, score)
    return df

# using spleeter library, split a audio file in 2 : vocal and background
def spleet(scene):
    """
    Parameters:
        str scene : nom de la scène (ex : '100_1')
    Returns:
        create 3 audio files : 'vocals.wav', 'vocals_resample.wav' and 'accompaniment.wav' using spleeter
    """
    name = scene + audio_extension.split('.')[-2]
    rep = spleeter_output_dir + name + '/'

    audiofile = dir_audio + scene + audio_extension
    audio_dirs  = [f for f in os.listdir(spleeter_output_dir)
                   if os.path.isdir(os.path.join(spleeter_output_dir, f))]

    if name not in audio_dirs :
        print("speeting", scene, "...")
        subprocess.run(['spleeter','separate','-i',audiofile,'-o',spleeter_output_dir])

    if 'vocals_resample.wav' not in os.listdir(rep):
        audiofile = rep + 'vocals.wav'
        signal, fe = librosa.load(audiofile, sr=samplerate, mono=True)
        filename = rep + 'vocals_resample.wav'
        sf.write(filename, signal, samplerate=fe, subtype='PCM_16')